In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

# Load dataset
df = pd.read_csv("/kaggle/input/datas/Crop_recommendation.csv")

In [1]:
# Separate features and labels
X_num = df[["temperature", "humidity", "ph", "water availability"]]

# Encode season
season_enc = OneHotEncoder(sparse=False)
X_season = season_enc.fit_transform(df[["season"]])

# Combine numeric + season
import numpy as np
X = np.hstack([X_num.values, X_season])

# Encode labels
label_enc = LabelEncoder()
y = label_enc.fit_transform(df["label"])

# One-hot target
y = tf.keras.utils.to_categorical(y)

# Normalize numeric features
scaler = StandardScaler()
X[:, :4] = scaler.fit_transform(X[:, :4])

NameError: name 'df' is not defined

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation="relu", input_shape=(X.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(y.shape[1], activation="softmax")
])

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Train
history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=16)

# Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")

# Example prediction
sample = np.array([[28, 80, 6.5, 300]])  # numeric values
sample_season = season_enc.transform([["rainy"]])
sample_input = np.hstack([sample, sample_season])
sample_input[:, :4] = scaler.transform(sample_input[:, :4])

pred = model.predict(sample_input)
print("Predicted Crop:", label_enc.inverse_transform([np.argmax(pred)]))